### Post writer in a LinkedIn style
**Olga Mondrus**



In [1]:
%%capture

!pip install langgraph
!pip install langchain_openai

In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Nebius_api_key.txt to Nebius_api_key.txt


# Simple LLM call (baseline)

Here, I implement a LangGraph graph with a single node that calls an LLM to write a LinkedIn post about a given topic.

In [109]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
import os
with open("Nebius_api_key.txt", "r") as f:
    nebius_api_key = f.read().strip()

os.environ["nebius_key"] = nebius_api_key

model = 'Qwen/Qwen3-235B-A22B'  # reasoning model, generates <think> .. </think> before the actual result
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=nebius_api_key,
)
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are an experienced LinkedIn content creator."},  # Setting the LLM's role
        {"role": "user", "content": "Write a concise and funny LinkedIn post about the benefits of eating chocolate when coding. Base it on articles and data science."}
    ],
)

In [110]:
print(response.choices[0].message.content)

<think>
Okay, the user wants a LinkedIn post about the benefits of eating chocolate while coding, backed by articles and data science, and it should be concise and funny. Let me start by recalling what I know about chocolate and cognitive benefits.

First, I need some credible sources. I remember a study from the University of Nottingham about cocoa enhancing blood flow to the brain. Also, flavonoids in dark chocolate might improve cognitive function. There's also the endorphin release aspect from the chocolate itself. For the data science angle, maybe mention that data shows productivity boosts or error reduction when snacking on chocolate.

I should structure the post to hook with humor, mention the science, add some stats, and end with a networking call-to-action. Maybe start with a joke about bugs in code and bugs in chocolate. Use emojis to keep it light and engaging. Check for any recent studies in 2023 to cite, but if not, stick to well-known research. Avoid being too technical;

In [111]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain, PromptTemplate
from langgraph.graph import StateGraph, START, END

import os

with open("Nebius_api_key.txt", "r") as f:
    nebius_api_key = f.read().strip() # added .strip() otherwise doesn't work

os.environ["NEBIUS_API_KEY"] = nebius_api_key

# prompt template
template = """You are an experienced LinkedIn content creator.
###
Write a concise and funny Linkedin post about the benefits of eating chocolate when coding. Base it on articles and data science.
{topic}"""
prompt = PromptTemplate(input_variables=["topic"], template=template)

llm = ChatOpenAI(
    model_name="Qwen/Qwen3-235B-A22B",
    temperature=0,
    base_url="https://api.studio.nebius.ai/v1/",
    openai_api_key=nebius_api_key,
)

# create an LLMChain instance
llm_chain = LLMChain(llm=llm, prompt=prompt)

# build the graph using StateGraph
workflow = StateGraph(str)

workflow.add_node('linkedin_post_writer', llm_chain.invoke)

workflow.add_edge(START, 'linkedin_post_writer')
workflow.add_edge('linkedin_post_writer', END)

app = workflow.compile()

topic = ""
response = app.invoke(topic)

In [112]:
print(app)

In [113]:
print(app.get_graph().draw_ascii())

      +-----------+      
      | __start__ |      
      +-----------+      
            *            
            *            
            *            
+----------------------+ 
| linkedin_post_writer | 
+----------------------+ 
            *            
            *            
            *            
      +---------+        
      | __end__ |        
      +---------+        


In [114]:
print(response['text'])

<think>
Okay, the user wants a LinkedIn post about the benefits of eating chocolate while coding, and they want it to be concise and funny. They also mentioned basing it on articles and data science. Let me start by brainstorming some key points.

First, I need to find some credible sources or studies that link chocolate consumption to cognitive benefits. Maybe dark chocolate has antioxidants or something that helps with focus. I remember reading that flavonoids in chocolate can improve blood flow to the brain. That's a good point.

Also, stress relief is important. Coding can be stressful, and chocolate might help reduce stress hormones. There's probably a study on that. Maybe mention how taking breaks with chocolate can prevent burnout.

Energy boost from the sugar and caffeine in chocolate? That's a quick pick-me-up during long coding sessions. But should I differentiate between types of chocolate? Dark vs. milk? Maybe suggest dark chocolate for higher cocoa content and less sugar.


In [118]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain, PromptTemplate
from langgraph.graph import StateGraph, START, END
import os

template = """You are an experienced LinkedIn content creator.
###
Write a concise and funny Linkedin post about the benefits of eating chocolate when coding. Base it on articles and data science.
{topic}"""
prompt = PromptTemplate(input_variables=["topic"], template=template)

# Initialize the LLM with the Nebius API configuration
llm = ChatOpenAI(
    model_name="Qwen/Qwen3-235B-A22B",
    temperature=0,
    base_url="https://api.studio.nebius.ai/v1/",
    openai_api_key=nebius_api_key,
)

# Create an LLMChain instance
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Build the graph
workflow = StateGraph(str) # Corrected to StateGraph

workflow.add_node('linkedin_post_writer', llm_chain.invoke)

workflow.add_edge(START, 'linkedin_post_writer')
workflow.add_edge('linkedin_post_writer', END)

app = workflow.compile()

# Run the chain with your topic
topic = ""
response = app.invoke(topic)

In [119]:
print(response['text'])

<think>
Okay, the user wants a LinkedIn post about the benefits of eating chocolate while coding, and they want it to be concise and funny. They also mentioned basing it on articles and data science. Let me start by brainstorming some key points.

First, I need to find some credible sources or studies that link chocolate consumption to cognitive benefits. I remember reading that dark chocolate contains flavonoids which can improve brain function. Maybe there's a study from a reputable university or a publication like the Journal of Neuroscience. Also, caffeine and theobromine in chocolate can boost alertness, which is useful during long coding sessions.

Next, the humorous angle. Programmers often joke about their caffeine addiction, so comparing chocolate to coffee could be funny. Maybe something about debugging code with chocolate or chocolate helping with those 3 AM coding marathons. Puns related to "sweet code" or "compiling smoothly" might work.

I should structure the post to sta

## The same method but a bit more beautiful

In [120]:
from langchain.prompts import ChatPromptTemplate
from typing import TypedDict
class GraphState(TypedDict):
    linkedin_post: str
    topic: str

def linkedin_post_writer(state: GraphState):
  topic = state.get('topic', '') # here, I extract the topic from the state
  llm = ChatOpenAI(
    model_name="Qwen/Qwen3-235B-A22B",
    temperature=0,
    base_url="https://api.studio.nebius.ai/v1/",
    openai_api_key=nebius_api_key,
  )
  template = """You are an experienced LinkedIn content creator.
  ###
  Write a concise and funny Linkedin post about the benefits of eating chocolate when coding. Base it on articles and data science.
  {topic}"""

  prompt_template = ChatPromptTemplate.from_template(template)
  prompt = prompt_template.invoke({'topic': topic})
  llm_output = llm.invoke(prompt).content
  # try to debug and print statement to see the output before returning
  print(f"--- Debug: linkedin_post_writer returning: {llm_output}")
  # return a dictionary with the specified key to update the state
  return {"linkedin_post": llm_output}

In [121]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, List

workflow = StateGraph(GraphState)

workflow.add_node('linkedin_post_writer', linkedin_post_writer)

workflow.add_edge(START, 'linkedin_post_writer')
workflow.add_edge('linkedin_post_writer', END)

app = workflow.compile()

In [122]:
!pip install grandalf

In [123]:
print(app.get_graph().draw_ascii())

      +-----------+      
      | __start__ |      
      +-----------+      
            *            
            *            
            *            
+----------------------+ 
| linkedin_post_writer | 
+----------------------+ 
            *            
            *            
            *            
      +---------+        
      | __end__ |        
      +---------+        


In [124]:
import re
response = app.invoke({"topic": "google"})
cleaned_response = re.sub(r'<think>.*?</think>', '', response["linkedin_post"], flags=re.DOTALL)
print(cleaned_response)

--- Debug: linkedin_post_writer returning: <think>
Okay, the user wants a LinkedIn post about the benefits of eating chocolate while coding, based on articles and data science. Let me start by recalling some studies I know. There's research on dark chocolate improving cognitive function because of flavonoids. Maybe mention improved focus or memory.

I should check if there's data on productivity and chocolate. Maybe a study where participants performed better on tasks after consuming cocoa. Also, stress reduction is a plus; programmers often deal with stress, so chocolate lowering cortisol could be a point.

Need to make it funny. Use coding puns. Like "debugging with a difference" or "sweeten the deal with clients." Maybe compare chocolate to a bug-free code – dark chocolate being the premium version.

Structure: Start with a catchy headline. Then a few bullet points with scientific references but in a light-hearted way. Add emojis for engagement. End with a call to action, like askin

# Workflow with steps

Here, I add steps to the workflow, such that it generates a LinkedIn post on a given topic based on scraped news articles from TechCruch.

For that, :
1. given a topic, I retrieve a relevant URL from TechCrunch
2. Given a URL, I retrieve the text
3. I convert the given text into a LinkedIn post

In [125]:
import requests
from bs4 import BeautifulSoup
index_url = 'https://techcrunch.com/2024/12/05/planet-a-foods-nabs-30m-to-make-tons-more-cocoa-free-chocolate/' #'https://techcrunch.com/2025/05/08/'

In [126]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
import os
import requests
from bs4 import BeautifulSoup
from typing import TypedDict, Annotated, List # Import necessary types

# --- Define the Graph State ---
class GraphState(TypedDict):
    """Represents the state of our multi-step graph."""
    topic: str # The initial topic for search fallback
    url: str # The retrieved article URL (or provided specific URL)
    article_text: str # The scraped article content
    linkedin_post: str # The final LinkedIn post
    specific_url: str # Optional: A specific URL to use instead of searching


# --- Setup API ---
with open("Nebius_api_key.txt") as f:
    nebius_api_key = f.read().strip()


# --- Node: Given a topic or specific URL, retrieve a relevant URL from TechCrunch ---
def get_techcrunch_url(state: GraphState) -> dict:
    """Uses a specific URL if provided, otherwise searches TechCrunch based on topic."""
    specific_url = state.get('specific_url')
    if specific_url:
        print(f"--- Debug: Using specific URL: {specific_url}")
        return {"url": specific_url}

    # what if no specific URL is provided
    topic = state.get('topic', '')
    if not topic:
        return {"url": "[ERROR] No topic or specific URL provided"}
    query = topic.replace(" ", "+")
    search_url = f"https://techcrunch.com/search/{query}"
    try:
        print(f"--- Debug: Searching TechCrunch for topic: {topic}")
        resp = requests.get(search_url, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")
        links = soup.find_all("a", href=True)
        article_urls = [
            link['href'] for link in links
            if link['href'].startswith("https://techcrunch.com/20")
        ]
        # Return the first relevant URL found (can be expanded later)
        if article_urls:
            print(f"--- Debug: Found URL via search: {article_urls[0]}")
            return {"url": article_urls[0]}
        else:
            return {"url": "[ERROR] No relevant articles found for topic"}
    except Exception as e:
        return {"url": f"[ERROR] Could not search TechCrunch: {e}"}

# --- Node: Given a URL, return the extracted article text ---
def scrape_techcrunch_article(state: GraphState) -> dict:
    """Extract article content from a TechCrunch URL and update state with text."""
    url = state.get('url', '')
    if not url or url.startswith("[ERROR]"):
        return {"article_text": f"[ERROR] Invalid or missing URL: {url}"}
    try:
        print(f"--- Debug: Attempting to scrape URL: {url}")
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        # Attempt to find article content using more general selectors
        article_content_div = soup.find("div", class_="article-content")
        if not article_content_div:
             article_content_div = soup.find("div", class_=lambda x: x and ("body-content" in x or "article" in x.lower())) # More flexible search
        if not article_content_div:
             article_content_div = soup.find("article")

        if not article_content_div:
             print(f"--- Debug: Could not find primary article content container on page: {url}")
             return {"article_text": "[ERROR] Could not find primary article content container on page"}

        # Extract all text content from the identified container
        text = article_content_div.get_text(separator="\n", strip=True) # Extract all text, using newline as separator

        if not text:
             print(f"--- Debug: Could not extract any text from content container: {url}")
             return {"article_text": "[ERROR] Could not extract any text from content container"}

        print(f"--- Debug: Successfully scraped text (first 200 chars): {text[:200]}...")
        return {"article_text": text[:4000]} # Trim for token limits, slightly increased

    except Exception as e:
        print(f"--- Debug: Error scraping URL {url}: {e}")
        return {"article_text": f"[ERROR] Could not scrape URL: {e}"}

# --- Node: Convert the given text into a LinkedIn post ---
def linkedin_post_writer(state: GraphState) -> dict:
    """Create a witty and concise LinkedIn post based on article text and update state."""
    article_text = state.get('article_text', '')
    if not article_text or article_text.startswith("[ERROR]"):
        print(f"--- Debug: Cannot write post, invalid article_text: {article_text}")
        return {"linkedin_post": f"[ERROR] Cannot write post: {article_text}"}

    llm = ChatOpenAI(
        model_name="Qwen/Qwen3-235B-A22B",
        temperature=0,
        base_url="https://api.studio.nebius.ai/v1/",
        openai_api_key=nebius_api_key,
    )
    template = """You are an experienced LinkedIn content creator.
    ###
    Write a concise and funny Linkedin post based on the following article content.
    Focus on key insights and keep it witty.
    Content: {article_text}""" # Use article_text from state

    prompt = ChatPromptTemplate.from_template(template).invoke({'article_text': article_text})
    llm_output = llm.invoke(prompt).content
    print(f"--- Debug: LLM output (first 200 chars): {llm_output[:200]}...")
    return {"linkedin_post": llm_output}

# --- Build the graph ---
workflow = StateGraph(GraphState) # Pass the defined state schema

workflow.add_node("get_techcrunch_url", get_techcrunch_url)
workflow.add_node("scrape_techcrunch_article", scrape_techcrunch_article)
workflow.add_node("linkedin_post_writer", linkedin_post_writer)

# Define the flow
workflow.set_entry_point("get_techcrunch_url")
workflow.add_edge("get_techcrunch_url", "scrape_techcrunch_article")
workflow.add_edge("scrape_techcrunch_article", "linkedin_post_writer")
workflow.set_finish_point("linkedin_post_writer")

# --- Compile and run ---
app = workflow.compile()

result = app.invoke({"specific_url": index_url, "topic": "how chocolate boosts focus for developers"})

--- Debug: Using specific URL: https://techcrunch.com/2024/12/05/planet-a-foods-nabs-30m-to-make-tons-more-cocoa-free-chocolate/
--- Debug: Attempting to scrape URL: https://techcrunch.com/2024/12/05/planet-a-foods-nabs-30m-to-make-tons-more-cocoa-free-chocolate/
--- Debug: Successfully scraped text (first 200 chars): Image Credits:
Planet A Foods
Climate
Planet A Foods nabs $30M to make tons more cocoa-free chocolate
Natasha Lomas
3:00 PM PST · December 5, 2024...
--- Debug: LLM output (first 200 chars): <think>
Okay, I need to create a concise and funny LinkedIn post based on the given article. Let me start by understanding the key points. The article is about Planet A Foods securing $30 million to p...


In [127]:
print(app.get_graph().draw_ascii())

        +-----------+          
        | __start__ |          
        +-----------+          
              *                
              *                
              *                
    +--------------------+     
    | get_techcrunch_url |     
    +--------------------+     
              *                
              *                
              *                
+---------------------------+  
| scrape_techcrunch_article |  
+---------------------------+  
              *                
              *                
              *                
  +----------------------+     
  | linkedin_post_writer |     
  +----------------------+     
              *                
              *                
              *                
         +---------+           
         | __end__ |           
         +---------+           


In [128]:
result['linkedin_post']

'<think>\nOkay, I need to create a concise and funny LinkedIn post based on the given article. Let me start by understanding the key points. The article is about Planet A Foods securing $30 million to produce more cocoa-free chocolate. The main elements here are the company name, the funding amount, the product (cocoa-free chocolate), and the implication that this is a response to climate issues since the article is under the "Climate" category.\n\nFirst, the user wants the post to be witty and focus on key insights. So I need to highlight the $30M funding and the cocoa-free aspect. Maybe play on the idea that chocolate without cocoa is a big deal, but also tie it to climate change. The challenge is making it funny without being too cheesy.\n\nPossible angles: The irony of chocolate without cocoa, the scale of production ("tons more"), the environmental angle. Maybe use chocolate-related puns. Also, the image credits to Planet A Foods, so maybe mention them in a humorous way.\n\nAvoid 

In [129]:
cleaned_response = re.sub(r'<think>.*?</think>', '', result['linkedin_post'], flags=re.DOTALL)
print(cleaned_response)



**LinkedIn Post:**  

🚨 *Breaking: Chocolate’s midlife crisis solved!* 🍫✨  
Planet A Foods just scored **$30M** to make *tons* more **cocoa-free chocolate**—because even cocoa beans need a sustainability intervention. 🌱  

Yes, you read that right: *Chocolate without the cocoa*. (Cue the existential crisis for traditional cacao farmers… and the world’s collective “wait, really?” 🤯)  

Why? Because deforestation > dessert, and someone finally realized we can have our chocolate *and* save the planet. 🌍✨  

**Key takeaway:** The future of food is weird, wonderful, and weirdly wonderful. 🚀  

Who’s first in line for a guilt-free Kit Kat? 🙋♂️  
#ClimateTech #SustainableChocolate #FoodInnovation #CocoaFreeAndFABULOUS  

*(P.S. Cocoa beans, we hardly knew ye. 😂)*  

**Tagging:** @Planet A Foods  

---  
*Short, snappy, and sprinkled with humor—because climate solutions should be as digestible as chocolate. 🍫*


# ReAct Framework
1. I convert the workflow into a ReAct agent; change the URL tool to return all the URLs (and not only the first one), so the agent will choose the best one.
2. I try to add memory (enabling interactive mode).

In [78]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

def linkedin_post_writer(topic: str):
    llm = ChatOpenAI(
        model_name="Qwen/Qwen3-235B-A22B",
        temperature=0,
        base_url="https://api.studio.nebius.ai/v1/",
        openai_api_key=nebius_api_key,
    )
    template = """You are an experienced LinkedIn content creator.
    ###
    Write a concise and funny Linkedin post based on the following article content.
    Focus on key insights and keep it witty.
    Content: {topic}"""
    prompt = ChatPromptTemplate.from_template(template).invoke({'topic': topic})
    return {"linkedin_post": llm.invoke(prompt).content}

# --- Node: Given a URL, return the extracted article text ---
def scrape_techcrunch_article(url: str):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract article content heuristically
        article = soup.find("article")
        if not article:
            article = soup.find("div", {"class": "article-content"})

        text = " ".join([p.get_text(strip=True) for p in article.find_all("p")])
        return {"article_text": text}
    except Exception as e:
        return {"article_text": f"[ERROR] Could not scrape URL: {e}"}


In [84]:
!pip install -U langgraph langchain

In [130]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from bs4 import BeautifulSoup
import requests

# --- Load Nebius key ---
with open("Nebius_api_key.txt") as f:
    nebius_api_key = f.read().strip()

# --- Shared LLM ---
llm = ChatOpenAI(
    model_name="Qwen/Qwen3-235B-A22B",
    temperature=0,
    base_url="https://api.studio.nebius.ai/v1/",
    openai_api_key=nebius_api_key,
)

# --- TOOL 1: Get URLs from TechCrunch search ---
def get_techcrunch_urls(topic: str) -> list[str]:
    """Search TechCrunch for articles related to a topic and return ALL article URLs."""
    query = topic.replace(" ", "+")
    search_url = f"https://techcrunch.com/search/{query}"
    try:
        resp = requests.get(search_url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(resp.text, "html.parser")

        # focus only on article title links
        links = soup.select("a.post-block__title__link")
        article_urls = [link['href'] for link in links if link['href'].startswith("https://techcrunch.com/20")]

        # Deduplicate
        seen, unique_urls = set(), []
        for url in article_urls:
            if url not in seen:
                seen.add(url)
                unique_urls.append(url)

        return unique_urls if unique_urls else ["[ERROR] No relevant articles found"]

    except Exception as e:
        return [f"[ERROR] Could not search TechCrunch: {e}"]

# --- TOOL 2: Scrape article ---
def scrape_article(url: str) -> str:
    """Extract article content from a TechCrunch URL."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        article_content_div = soup.find("div", class_="article-content")
        if not article_content_div:
            article_content_div = soup.find("div", class_=lambda x: x and ("body-content" in x or "article" in x.lower()))
        if not article_content_div:
            article_content_div = soup.find("article")

        if not article_content_div:
            return "[ERROR] Could not find article content"

        text = article_content_div.get_text(separator="\n", strip=True)
        return text[:3000] if text else "[ERROR] Could not extract text"
    except Exception as e:
        return f"[ERROR] Could not scrape article: {e}"

# --- TOOL 3: Create LinkedIn post ---
def linkedin_post_writer(article_text: str) -> str:
    """Create a witty LinkedIn post based on article text."""
    template = """You are an experienced LinkedIn content creator.
###
Write a concise and funny LinkedIn post based on the following article content.
Focus on key insights and keep it witty.
Content: {topic}"""
    prompt = ChatPromptTemplate.from_template(template).invoke({'topic': article_text})
    return llm.invoke(prompt).content

# --- Define tools ---
tools = [
    Tool.from_function(
        func=get_techcrunch_urls,
        name="search_techcrunch",
        description="Search TechCrunch for articles related to a topic and return ALL article URLs."
    ),
    Tool.from_function(
        func=scrape_article,
        name="scrape_article",
        description="Extract article content from a TechCrunch URL."
    ),
    Tool.from_function(
        func=linkedin_post_writer,
        name="linkedin_post_writer",
        description="Create a witty and concise LinkedIn post based on article text."
    ),
]

# --- Add memory for interactive mode ---
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# --- Build the ReAct Agent ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Classic ReAct agent
    verbose=True,
    memory=memory
)

# --- Example usage ---
result = agent.invoke({"input": "Create a LinkedIn post about how chocolate boosts coding productivity using a recent TechCrunch article."})

print("✅ Final Output:\n")
print(result["output"])




> Entering new AgentExecutor chain...
<think>
Okay, I need to create a LinkedIn post about how chocolate boosts coding productivity using a recent TechCrunch article. Let me start by figuring out the steps I need to take.

First, I should search TechCrunch for articles related to chocolate and coding productivity. The user provided the search_techcrunch tool, so I'll use that with the topic "chocolate coding productivity". Let me check what comes up.

After getting the search results, I need to pick the most relevant article. Once I have the URL, I'll use the scrape_article tool to extract the content. Then, using that content, I'll use the linkedin_post_writer to generate a witty and concise post. 

Wait, but the search might return multiple articles. I need to make sure I choose the most recent one as per the question. Let me first perform the search and see what URLs are available. If there are multiple, I'll select the latest one. Then proceed to scrape it. Once I have the articl

In [131]:
!pip install langsmith

In [132]:
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
try:
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
except userdata.SecretNotFoundError:
    print("LANGCHAIN_API_KEY not found in Colab Secrets. Please add it.")
    # You may want to handle this error or exit if the key is essential

os.environ["LANGCHAIN_PROJECT"] = "GenAI-Research-Demo"

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Define prompt (like your LinkedIn postwriter)
prompt = ChatPromptTemplate.from_template(
    "Write a concise and witty LinkedIn post about: {topic}"
)

llm = ChatOpenAI(
    model_name="Qwen/Qwen3-235B-A22B",
    temperature=0,
    base_url="https://api.studio.nebius.ai/v1/",
    openai_api_key=nebius_api_key,
)


chain = LLMChain(prompt=prompt, llm=llm)

# Run with tracing
response = chain.invoke({"topic": "chocolate boosts coding productivity"})
print(response['text'])

<think>
Okay, the user wants a LinkedIn post about how chocolate boosts coding productivity. Let me start by thinking about the key points. First, I need to connect chocolate with coding. Maybe talk about the science behind chocolate, like caffeine and theobromine improving focus. Also, the sugar rush providing quick energy. But I should keep it light and witty, not too technical.

Who is the audience here? Professionals, developers, maybe tech-savvy people who use LinkedIn. They'd appreciate humor but also some credible info. Need to make it relatable. Maybe mention common coding scenarios, like debugging or late-night coding sessions.

Should I include some stats or studies? Maybe a playful mention of research without getting too detailed. Emphasize that it's a fun take but still has truth. Also, add emojis to make it visually engaging on LinkedIn. Chocolate and computer/code-related emojis.

Structure-wise, start with a catchy headline. Something like "The Secret Ingredient..." Then

In [133]:
import langsmith
print("LangSmith key loaded?", os.environ.get("LANGCHAIN_API_KEY") is not None)
print("Tracing active:", os.environ.get("LANGCHAIN_TRACING_V2"))

LangSmith key loaded? True
Tracing active: true


In [134]:
import os
print("Tracing active:", os.environ.get("LANGCHAIN_TRACING_V2"))
print("Project:", os.environ.get("LANGCHAIN_PROJECT"))

Tracing active: true
Project: GenAI-Research-Demo
